In [1]:
import pandas as pd
import random
import io
import ast
import numpy as np
import os
import re
import string
import pickle
import copy
import torch.nn.functional as functional
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from bs4 import BeautifulSoup
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import nltk
from nltk.stem import WordNetLemmatizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import pairwise_distances

In [116]:
random.seed(13)
np.random.seed(13)
torch.manual_seed(13)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(13)
torch.cuda.manual_seed(13)

### Loading texts, split on train, val and test


In [3]:
!wget https://www.dropbox.com/s/n8nfxxh5azocrrm/train_LASER.csv?dl=0
#!wget https://www.dropbox.com/s/m4t5pgqpjwjjyj2/X_train_LASER.npy?dl=0   #Матрица X_train уже посчитанная
!wget https://www.dropbox.com/s/je9am5c77ytfcaf/test_LASER.csv?dl=0

--2020-04-20 11:22:47--  https://www.dropbox.com/s/n8nfxxh5azocrrm/train_LASER.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n8nfxxh5azocrrm/train_LASER.csv [following]
--2020-04-20 11:22:48--  https://www.dropbox.com/s/raw/n8nfxxh5azocrrm/train_LASER.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbaca06ce32a87c4d1aa8b16042.dl.dropboxusercontent.com/cd/0/inline/A2MvXOOayRlhDHEeb5U7AEDl4G0ISYIvm886CCad_BokBGbvrxzqFTDF6oFmkwqfO2_1TMTUD5ik34OwRt8l7InRz3ml-897Dxu4BIAd0YeXC6x1mm-fDsxBdesShHj5HX0/file# [following]
--2020-04-20 11:22:48--  https://ucbaca06ce32a87c4d1aa8b16042.dl.dropboxusercontent.com/cd/0/inline/A2MvXOOayRlhDHEeb5U7AEDl4G0ISYIvm886CCad_BokBGbvrxzqFTDF6oFmkwqfO2_1TMTUD5ik34OwRt8l7InRz3ml-897Dxu4

In [4]:
#открываем файл с train dataset
df_train = pd.read_csv("train_LASER.csv?dl=0").dropna()
train_texts = list(df_train['abstracts'])
train_labels = list(df_train['labels'])

len(train_texts)

80948

In [5]:
#открываем файл с test_dataset
df_test = pd.read_csv("test_LASER.csv?dl=0").dropna()
test_texts = list(df_test['lemm_abstracts'])              #лемм там нет, просто название неправильное не дала
test_labels = list(df_test['labels'])

len(test_texts)

10150

In [6]:
test_texts[1000]

'использование биотехнология, медицинская микробиология и может быть использовано в производстве эубиотических препаратов и бифидосодержащих продуктов питания. сущность изобретения питательная среда в качестве основы содержит молочную сыворотку, осветленную последовательной обработкой обезжиренного молока кислотой и щелочью, и дрожжевой автолизат сухой или жидкий и дополнительно натрий фосфорнокислый однозамещенный двуводный, аммоний хлористый и магний сернокислый семиводный, при следующем соотношении компонентов мас. молочная сыворотка осветленная , , , лактоза , , , дрожжевой автолизат сухой или жидкий , , и , , , d,l цистеин , , , nah po • h o , , , nh cl , , , mgso • h o , , , агар агар , , , вода остальное, ph , , .'

In [7]:
#разбиваем на train и val английские тексты
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, \
                                                          test_size=0.15, shuffle = True, random_state=42, stratify=train_labels)

In [8]:
print(len(train_labels))
print(len(val_labels))

68805
12143


Нужно почистить тексты

In [9]:
def cleaning(text):
    clean_txt = []
    for sentence in text.split('.'):    #[:-1], чтобы не было последней пустой строки 
        sentence = re.sub(r'[•⋅−]', ' ', sentence)
        pattern = re.compile(r'(\s,){2,}')                 # регулярка для того чтобы несколько подряд запятых заменять на одну
        sentence = re.sub(pattern, ', ', sentence)
        sent = sentence.strip()
        sent = re.sub(r'( )+', ' ', sent)
        sent = re.sub(' ,', ',', sent)
     #   if len(sent.split()) > 2 and sent != 'ф лы, ил':
        if len(sent.split()) > 2 and 'ф лы' not in sent:
             clean_txt.append(sent.lstrip(',').rstrip(',').strip())
    return '. '.join(clean_txt)         

In [10]:
test_texts = [cleaning(text) for text in test_texts]
val_texts = [cleaning(text) for text in val_texts]
train_texts = [cleaning(text) for text in train_texts]

In [11]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
#add_labels = label_encoder.fit_transform(df_add['us_code'])
y_train[:10]

array([ 58, 199, 218,   7, 196, 115,  69,   1,  13,  23])

In [12]:
y_val = label_encoder.transform(val_labels)
y_val[:10]

array([ 23,  82,  57, 164,  99, 146,  24, 161, 194,  16])

In [13]:
y_test = label_encoder.transform(test_labels)

In [14]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(f'Using device: {DEVICE}')

Using device: cuda


### Loading LASER embeddings

In [15]:
!pip install laserembeddings

     |████████████████████████████████| 859 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 3.1 MB/s  eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883999 sha256=e69dfad0ac4755f8ea15c4b6c7090b7a7d04eb146f21b4ba67d37fee366202dc
  Stored in directory: /root/.cache/pip/wheels/cc/1c/a7/137ec649e0276d453fd870aba3b43e0734d087406a33ce7596
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.38
    Uninstalling sacremoses-0.0.38:
      Successfully uninstalled sacremoses-0.0.38


In [16]:
!python -m laserembeddings download-models
from laserembeddings import Laser

laser = Laser()

embeddings = laser.embed_sentences(train_texts[:100], lang='en')
embeddings.shape


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


(100, 1024)

In [19]:
laser.embed_sentences('an original image at high speed.', lang='en') == laser.embed_sentences('An original image at high speed', lang='en') 

array([[False, False, False, ..., False, False, False]])

In [20]:
laser1 = laser.embed_sentences('Hello world', lang='en')
laser2 = laser.embed_sentences('hello world', lang='en')
laser1 == laser2

array([[ True,  True,  True, ...,  True,  True,  True]])

### Texts vectorization

In [21]:
lens_train = [len(text.split()) for text in train_texts]
np.min(lens_train)

5

In [23]:
X_train = laser.embed_sentences(train_texts, lang='en') 
#X_train = np.load('X_train_LASER.npy?dl=0')
X_train.shape

(68805, 1024)

In [24]:
X_val = laser.embed_sentences(val_texts, 'en')
X_val.shape

(12143, 1024)

In [25]:
X_test = laser.embed_sentences(test_texts, 'ru')
X_test.shape

(10150, 1024)

### Experiments

#### SVC Classifier

In [26]:
def predict(y_true, y_pred):
    print('micro: ', f1_score(y_true, y_pred, average='micro')*100)
    print('weighted: ', f1_score(y_true, y_pred, average='weighted')*100)
    

In [27]:
from sklearn import linear_model
alphas = [0.01, 0.001, 0.0001, 0.00001]
for value in alphas:
    print('alpha=', value)
    clf = linear_model.SGDClassifier(max_iter=10000, tol=1e-3, early_stopping=True, random_state=42, verbose=0,\
                                     learning_rate='adaptive', eta0 = 1e-2, n_jobs=-1, alpha=value)
    clf.fit(X_train, y_train)
    print('f1_train:' + '\n')
    predict(y_train, clf.predict(X_train))
    print('f1_val:' + '\n')
    predict(y_val, clf.predict(X_val))
    print('f1_test:' + '\n')
    predict(y_test, clf.predict(X_test))
    print('==========================')
    

alpha= 0.01
f1_train:

micro:  57.10631494804157
weighted:  56.5375505216593
f1_val:

micro:  43.71242691262456
weighted:  42.25084551411765
f1_test:

micro:  27.960591133004925
weighted:  27.86844528062134
alpha= 0.001
f1_train:

micro:  57.12084877552503
weighted:  56.53401786247757
f1_val:

micro:  43.90183644898295
weighted:  42.49525445473236
f1_test:

micro:  28.43349753694581
weighted:  28.005606335096395
alpha= 0.0001
f1_train:

micro:  56.33311532592108
weighted:  55.70234535342989
f1_val:

micro:  43.572428559664004
weighted:  42.049889347509094
f1_test:

micro:  28.778325123152708
weighted:  28.34019262102049
alpha= 1e-05
f1_train:

micro:  56.08458687595378
weighted:  55.418405310794974
f1_val:

micro:  43.77830849048834
weighted:  42.25464547491383
f1_test:

micro:  28.660098522167488
weighted:  28.10864120650415


####  Logistic regression and FFNN

In [28]:
from tqdm import tnrange
def iterate_minibatches(data, labels, batch_size=100, shuffle=True, verbose=True):
    indices = np.arange(len(data))
    if shuffle:
        indices = np.random.permutation(indices)        
    irange = tnrange if verbose else range
 #   print(indices)
    for start in irange(0, len(indices), batch_size):
        yield data[indices[start: start + batch_size]], labels[indices[start: start + batch_size]]

In [48]:
class LogisticRegression(nn.Module):
    def __init__(self, emb_dim, num_labels):
        super().__init__()
        self.linear = nn.Linear(emb_dim, num_labels)
        nn.init.xavier_normal_(self.linear.weight)

    def forward(self, emb):
        return F.log_softmax(self.linear(emb), dim=1)

In [43]:
#веса для лосс функции
distrib_class = dict(Counter(y_train))
values_class = np.zeros((len(distrib_class)))
for key in sorted(distrib_class.keys()):
    values_class[key] = distrib_class[key]
    
weights = torch.FloatTensor(np.min(values_class)/values_class).cuda()
weights

tensor([0.1667, 0.5060, 0.1673, 0.4942, 0.1778, 0.5215, 0.9444, 0.4670, 0.7025,
        0.7658, 0.4208, 0.5743, 0.5556, 0.1860, 0.7025, 0.2033, 0.1667, 0.1667,
        0.1667, 0.3160, 0.1673, 0.2058, 0.1667, 0.1667, 0.3427, 0.1852, 0.6746,
        0.4497, 0.7798, 0.8500, 0.1667, 0.1667, 0.1667, 0.6967, 0.1923, 0.2607,
        0.1667, 0.8095, 0.2429, 0.1683, 0.5152, 0.1667, 0.3696, 0.3899, 0.1667,
        0.6159, 0.4167, 0.4857, 0.7143, 0.1667, 0.5944, 0.4229, 0.3014, 0.7025,
        0.5060, 0.8763, 0.6391, 0.6159, 0.2415, 0.7589, 0.2673, 0.3427, 0.9659,
        0.4913, 0.2316, 0.5484, 0.3400, 0.5667, 0.1667, 0.1667, 0.5556, 0.2656,
        0.1667, 0.1690, 0.2632, 0.1667, 0.1667, 0.3571, 0.3617, 0.8173, 0.8333,
        0.2033, 0.1667, 0.1673, 0.2796, 0.6641, 0.1667, 0.2464, 0.3846, 0.1667,
        0.1667, 0.8947, 0.3069, 0.9770, 0.1667, 0.6028, 0.4126, 0.1667, 0.4913,
        0.2068, 0.3427, 0.4913, 0.6159, 0.7456, 0.2225, 0.5667, 0.9770, 0.7143,
        0.2450, 0.8173, 0.5556, 0.1667, 

In [118]:
N_EPOCHS = 600
LEARNING_RATE = 4e-3
model = LogisticRegression(1024,225).to(DEVICE) 
#model = FFNN(300).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)
#loss_function = nn.NLLLoss()
loss_function = nn.NLLLoss(weight=weights) #если с весами

In [95]:
def train_model_epoch(train_dataset, labels, model, optimizer, batch_size):
    model.train()
    train_loss = 0
    train_predictions = np.array([])
    pred_labels = np.array([])
    
    for texts, labs in iterate_minibatches(np.array(train_dataset), np.array(labels), batch_size=batch_size, shuffle=True, verbose=False):
        emb_ix = torch.tensor(texts, dtype=torch.float32).to(DEVICE) #cuda
     #   print(labels[i: i+batch_size])
        res_labels = torch.tensor(labs).to(DEVICE)     #cuda 
      #  print(labels.shape)
        optimizer.zero_grad()
        output = model(emb_ix)
 
        loss = loss_function(output, res_labels)
        train_loss += loss.item()
   #     train_acc += (output.argmax(1) == res_labels).sum().item()
      #  print(train_acc)
        
        y_pred = output.argmax(1).cpu().numpy()
        train_predictions = np.hstack((train_predictions, y_pred))
        pred_labels = np.hstack((pred_labels, res_labels.cpu().numpy()))
     #   print(train_f1)
     #   break
        loss.backward()
        optimizer.step()

    train_f1_micro = f1_score(pred_labels, train_predictions, average='micro')*100
    train_f1_weighted = f1_score(pred_labels, train_predictions, average='weighted')*100

    return train_loss / (len(train_dataset)//batch_size), train_f1_micro, train_f1_weighted

In [86]:
def test(test_dataset, labels, model, batch_size):
    model.eval()
#    acc = 0
    test_predictions = np.array([])
    test_loss = 0
    for texts, labs in iterate_minibatches(np.array(test_dataset), np.array(labels), batch_size=batch_size, shuffle=False, verbose=False):
        with torch.no_grad():
            emb_ix = torch.tensor(texts, dtype = torch.float32).to(DEVICE) #cuda
            res_labels = torch.tensor(labs).to(DEVICE)   #cuda
        #  print(labels.shape)
            output = model(emb_ix)
            y_pred = output.argmax(1).cpu().numpy()
            loss = loss_function(output, res_labels)
            test_loss += loss.item()
            test_predictions = np.hstack((test_predictions, y_pred))

    test_f1_micro = f1_score(labels, test_predictions, average='micro')*100
    test_f1_weighted = f1_score(labels, test_predictions, average='weighted')*100


    return test_loss / (len(test_dataset)//batch_size), test_f1_micro, test_f1_weighted

In [119]:
print('Training...')
best_loss = 10
max_step = 2
step=0
for epoch in tnrange(N_EPOCHS+1):
    train_loss, train_f1_micro, train_f1_weighted = train_model_epoch(X_train, y_train, model, optimizer, batch_size=300)
 #   train_loss, train_acc = train_model_epoch_sample(train_texts, labels, model, optimizer)
    val_loss, val_f1_micro, val_f1_weighted = test(X_val, y_val, model, batch_size=300)
    if val_loss > best_loss:
      step += 1
      
      if step > max_step:
        break
    else:
      best_loss = val_loss
      step = 0
      weights_model = copy.deepcopy(model.state_dict())
           
    
      new_model = LogisticRegression(1024,225).to(DEVICE)
      new_model.load_state_dict(weights_model)

    if epoch % 10 == 0:
     #   test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=300)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
     #   print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {val_f1_micro:.4f} | Val f1 weighted: {val_f1_weighted:.4f}')
print('Finished training.')

Training...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """



Train loss: 5.27935783 | Train f1 micro: 4.5709  | Train f1 weighted: 5.0817

Val f1 micro: 10.7387 | Val f1 weighted: 9.6471

Train loss: 3.46638404 | Train f1 micro: 41.3473  | Train f1 weighted: 40.0220

Val f1 micro: 36.6137 | Val f1 weighted: 35.2673

Train loss: 2.85887246 | Train f1 micro: 46.1289  | Train f1 weighted: 45.4940

Val f1 micro: 39.4631 | Val f1 weighted: 38.8725

Train loss: 2.56405438 | Train f1 micro: 48.8627  | Train f1 weighted: 48.4498

Val f1 micro: 41.1513 | Val f1 weighted: 40.7178

Train loss: 2.39116531 | Train f1 micro: 50.5312  | Train f1 weighted: 50.2058

Val f1 micro: 41.6866 | Val f1 weighted: 41.4617

Train loss: 2.28076462 | Train f1 micro: 51.6372  | Train f1 weighted: 51.3668

Val f1 micro: 42.3866 | Val f1 weighted: 42.0252

Train loss: 2.20221206 | Train f1 micro: 52.5936  | Train f1 weighted: 52.3515

Val f1 micro: 42.3289 | Val f1 weighted: 42.2653

Train loss: 2.14467519 | Train f1 micro: 53.4234  | Train f1 weighted: 53.1475

Val f1 micro

In [91]:
def compute_f1(model, data, labels):
    model.eval()
    emb_ix = torch.tensor(data, dtype = torch.float32).to(DEVICE)
    output = new_model(emb_ix)
    y_pred = output.argmax(1).cpu().numpy()
    print('f1_micro= ', f1_score(labels, y_pred, average='micro')*100)
    print('f1_weighted= ', f1_score(labels, y_pred, average='weighted')*100)

In [101]:
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)

Train f1
f1_micro=  56.292420608967376
f1_weighted=  55.392060199545575
Val f1
f1_micro=  44.14065716873919
f1_weighted=  42.41864268725163
Test f1
f1_micro=  31.605911330049263
f1_weighted=  29.32432711795015


In [120]:
# С весами для каждого класса
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)

Train f1
f1_micro=  55.69653368214519
f1_weighted=  55.448268449347424
Val f1
f1_micro=  43.28419665650993
f1_weighted=  43.32324221800313
Test f1
f1_micro=  30.3743842364532
f1_weighted=  30.236726656124937


In [ ]:
y_pred[10:20]

In [ ]:
y_test[10:20]

In [121]:
new_model.eval()
emb_ix = torch.tensor(X_test, dtype = torch.float32).to(DEVICE)
output = new_model(emb_ix)
y_pred = output.argmax(1).cpu().numpy()

print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.5455    0.2400    0.3333        75
           1     0.3714    0.5200    0.4333        25
           2     0.6780    0.5333    0.5970        75
           3     0.2414    0.2692    0.2545        26
           4     0.3304    0.5211    0.4044        71
           5     0.1739    0.3333    0.2286        24
           6     0.1970    0.9286    0.3250        14
           7     0.1571    0.4231    0.2292        26
           8     0.3714    0.7222    0.4906        18
           9     0.0833    0.2941    0.1299        17
          10     0.1628    0.2333    0.1918        30
          11     0.7500    0.1364    0.2308        22
          12     0.5294    0.3913    0.4500        23
          13     0.7750    0.4627    0.5794        67
          14     0.2222    0.1111    0.1481        18
          15     0.4211    0.1290    0.1975        62
          16     0.2083    0.3333    0.2564        75
          17     0.4755    

In [106]:
class FFNN(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.linear1 = nn.Linear(emb_dim, 256)
        self.linear2 = nn.Linear(256, 225)
        self.relu = nn.ReLU()
   #     self.norm1 = nn.BatchNorm1d(256)
    #    self.dropout = nn.Dropout(0.1)
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)

    def forward(self, emb):
        output = self.linear2(self.relu(self.linear1(emb)))
      #  output = self.linear3(self.relu(self.norm2(output)))
        return F.log_softmax(output, dim=1)
      #  return output

In [125]:
N_EPOCHS = 200
LEARNING_RATE = 3e-4

model = FFNN(1024).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)
#loss_function = nn.NLLLoss()
loss_function = nn.NLLLoss(weight=weights)

In [126]:
print('Training...')
best_loss = 10
max_step = 2
step=0
for epoch in tnrange(N_EPOCHS+1):
    train_loss, train_f1_micro, train_f1_weighted = train_model_epoch(X_train, y_train, model, optimizer, batch_size=300)
 #   train_loss, train_acc = train_model_epoch_sample(train_texts, labels, model, optimizer)
    val_loss, val_f1_micro, val_f1_weighted = test(X_val, y_val, model, batch_size=300)
    if val_loss > best_loss:
      step += 1
      
      if step > max_step:
        break
    else:
      best_loss = val_loss
      step = 0
      weights_model = copy.deepcopy(model.state_dict())
           
    
      new_model = FFNN(1024).to(DEVICE)
      new_model.load_state_dict(weights_model)

    if epoch % 10 == 0:
     #   test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=300)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
     #   print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {val_f1_micro:.4f} | Val f1 weighted: {val_f1_weighted:.4f}')
print('Finished training.')

Training...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """



Train loss: 5.42312918 | Train f1 micro: 1.4912  | Train f1 weighted: 0.7027

Val f1 micro: 2.9070 | Val f1 weighted: 1.0756

Train loss: 3.94297387 | Train f1 micro: 19.2123  | Train f1 weighted: 17.1123

Val f1 micro: 18.9574 | Val f1 weighted: 17.0402

Train loss: 3.33030120 | Train f1 micro: 27.9922  | Train f1 weighted: 26.5352

Val f1 micro: 26.6162 | Val f1 weighted: 25.1320

Train loss: 2.98471640 | Train f1 micro: 32.7709  | Train f1 weighted: 31.7971

Val f1 micro: 30.9643 | Val f1 weighted: 30.0582

Train loss: 2.75667248 | Train f1 micro: 35.9988  | Train f1 weighted: 35.3208

Val f1 micro: 33.8137 | Val f1 weighted: 33.2047

Train loss: 2.58950234 | Train f1 micro: 38.5888  | Train f1 weighted: 38.0702

Val f1 micro: 35.4278 | Val f1 weighted: 34.7578

Train loss: 2.45712198 | Train f1 micro: 40.7558  | Train f1 weighted: 40.3176

Val f1 micro: 36.4407 | Val f1 weighted: 36.1856

Train loss: 2.35100836 | Train f1 micro: 42.4853  | Train f1 weighted: 42.1128

Val f1 micro:

In [124]:
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)

Train f1
f1_micro=  55.43056463919773
f1_weighted=  54.6505936129052
Val f1
f1_micro=  44.82417853907601
f1_weighted=  43.7256178489345
Test f1
f1_micro=  31.605911330049263
f1_weighted=  30.110759798519947


In [127]:
# c весами
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)


Train f1
f1_micro=  50.025434198096065
f1_weighted=  49.76002335644124
Val f1
f1_micro=  41.09363419253891
f1_weighted=  41.117561592509155
Test f1
f1_micro=  30.108374384236452
f1_weighted=  29.879534939631014


In [129]:
new_model.eval()
emb_ix = torch.tensor(X_test, dtype = torch.float32).to(DEVICE)
output = new_model(emb_ix)
y_pred = output.argmax(1).cpu().numpy()

print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.4800    0.3200    0.3840        75
           1     0.3415    0.5600    0.4242        25
           2     0.6852    0.4933    0.5736        75
           3     0.2963    0.3077    0.3019        26
           4     0.3542    0.4789    0.4072        71
           5     0.3600    0.3750    0.3673        24
           6     0.2692    0.5000    0.3500        14
           7     0.1447    0.4231    0.2157        26
           8     0.4242    0.7778    0.5490        18
           9     0.2222    0.4706    0.3019        17
          10     0.2069    0.4000    0.2727        30
          11     0.6000    0.1364    0.2222        22
          12     0.4444    0.3478    0.3902        23
          13     0.5686    0.4328    0.4915        67
          14     0.1250    0.0556    0.0769        18
          15     0.2564    0.1613    0.1980        62
          16     0.2336    0.3333    0.2747        75
          17     0.5079    

### Best model for classification on eight classes
Классификация на уровне раздела

In [130]:
train_labels = [label[0] for label in train_labels]
train_labels[:10]

['B', 'H', 'H', 'A', 'H', 'E', 'B', 'A', 'A', 'A']

In [131]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
#add_labels = label_encoder.fit_transform(df_add['us_code'])
y_train[:10]

array([1, 7, 7, 0, 7, 4, 1, 0, 0, 0])

In [133]:
val_labels = [label[0] for label in val_labels]
test_labels = [label[0] for label in test_labels]

In [134]:
y_val = label_encoder.transform(val_labels)
y_test = label_encoder.transform(test_labels)

In [143]:
distrib_class = dict(Counter(y_train))
values_class = np.zeros((len(distrib_class)))
for key in sorted(distrib_class.keys()):
    values_class[key] = distrib_class[key]
    
weights = torch.FloatTensor(np.min(values_class)/values_class).cuda()
weights

tensor([0.1296, 0.0792, 0.1756, 1.0000, 0.3634, 0.1159, 0.0994, 0.0936],
       device='cuda:0')

In [144]:
N_EPOCHS = 600
LEARNING_RATE = 4e-3
model = LogisticRegression(1024,8).to(DEVICE) 
#model = FFNN(300).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.NLLLoss(weight=weights)

In [145]:
print('Training...')
best_loss = 10
max_step = 2
step=0
for epoch in tnrange(N_EPOCHS+1):
    train_loss, train_f1_micro, train_f1_weighted = train_model_epoch(X_train, y_train, model, optimizer, batch_size=256)
 #   train_loss, train_acc = train_model_epoch_sample(train_texts, labels, model, optimizer)
    val_loss, val_f1_micro, val_f1_weighted = test(X_val, y_val, model, batch_size=256)
    if val_loss > best_loss:
      step += 1
      
      if step > max_step:
        break
    else:
      best_loss = val_loss
      step = 0
      weights_model = copy.deepcopy(model.state_dict())
           
    
      new_model = LogisticRegression(1024,8).to(DEVICE)
      new_model.load_state_dict(weights_model)

    if epoch % 10 == 0:
     #   test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=300)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
     #   print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {val_f1_micro:.4f} | Val f1 weighted: {val_f1_weighted:.4f}')
print('Finished training.')

Training...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """



Train loss: 1.86592713 | Train f1 micro: 37.5322  | Train f1 weighted: 36.8728

Val f1 micro: 46.7100 | Val f1 weighted: 45.2527

Train loss: 1.21636310 | Train f1 micro: 54.7853  | Train f1 weighted: 54.8369

Val f1 micro: 54.2617 | Val f1 weighted: 54.2241

Train loss: 1.11747842 | Train f1 micro: 57.7138  | Train f1 weighted: 57.7231

Val f1 micro: 56.9464 | Val f1 weighted: 57.1329

Train loss: 1.07044750 | Train f1 micro: 58.9289  | Train f1 weighted: 58.8919

Val f1 micro: 57.9840 | Val f1 weighted: 58.2294

Train loss: 1.04256399 | Train f1 micro: 59.5756  | Train f1 weighted: 59.5227

Val f1 micro: 58.8652 | Val f1 weighted: 58.9814

Train loss: 1.02248640 | Train f1 micro: 60.1206  | Train f1 weighted: 60.0614

Val f1 micro: 59.4005 | Val f1 weighted: 59.4919

Train loss: 1.00937777 | Train f1 micro: 60.4404  | Train f1 weighted: 60.3665

Val f1 micro: 59.1781 | Val f1 weighted: 59.2625
Finished training.


In [142]:
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)

Train f1
f1_micro=  63.72647336676114
f1_weighted=  63.71965529939381
Val f1
f1_micro=  61.97809437536029
f1_weighted=  61.95180651521428
Test f1
f1_micro=  61.10344827586207
f1_weighted=  60.9140182629637


In [146]:
#с весами
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)

Train f1
f1_micro=  60.32555773562967
f1_weighted=  60.08820233926087
Val f1
f1_micro=  59.07930494935354
f1_weighted=  58.974575493828695
Test f1
f1_micro=  56.91625615763547
f1_weighted=  57.610310375600285
